In [20]:
import pandas as pd

import sys

import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi

pd.set_option('display.max_columns', 500)

In [27]:
from nba_api.stats.static import players
from nba_api.stats.static import teams

# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
nba_teams = teams.get_teams()

nba_players = pd.DataFrame(nba_players)
nba_teams = pd.DataFrame(nba_teams)

active_plyaers = nba_players[nba_players['is_active']]

seasons = []
for i in list(range(1983,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]
    
### 
testplayer='203076'
testgame = '0022201200'
testteam = '1610612755'

In [132]:
from nba_api.stats.endpoints import leaguegamefinder

result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]

listframe = []
for i in seasons[:1]:
    playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
    playerfinder['season']=i
    listframe.append(playerfinder)
    time.sleep(.5)
allplayers = pd.concat(listframe,axis=0)

listframe = []
for i in seasons[:1]:
    gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_type_nullable="Regular Season",season_nullable=i).get_data_frames()[0]
    listframe.append(gamefinder)
    time.sleep(1)
gamehistory = pd.concat(listframe,axis=0)
gamehistory['GAME_ID'] = pd.to_numeric(gamehistory['GAME_ID'])
gamehistory['GAME_DATE'] = pd.to_datetime(gamehistory['GAME_DATE'])

In [29]:
bxv2 = nbaapi.boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=testgame).get_data_frames()


In [133]:
inseasondates = ['2023-11-03','2023-12-04','2023-12-05','2023-12-09']
gamehistory.loc[gamehistory['GAME_DATE'].isin(inseasondates),'INSEASON']=1
gamehistory.loc[(gamehistory['GAME_DATE'].dt.day.between(3,29)) & (gamehistory['GAME_DATE'].dt.month == 11) & (gamehistory['GAME_DATE'].dt.dayofweek.isin([1,4])),'INSEASON'] = 1
gamehistory[gamehistory['INSEASON']==1].to_csv('test.csv')
isgames = gamehistory[gamehistory['INSEASON']==1].GAME_ID.drop_duplicates()

In [134]:
isgames

22     22301201
23     22301203
26     22301202
27     22301204
98     22300057
         ...   
458    22300002
459    22300003
460    22300005
464    22300004
465    22300001
Name: GAME_ID, Length: 64, dtype: int64